In [1]:
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os
import wandb
from transformers import ViTMAEConfig

from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
from transformers.models.vit_mae.modeling_vit_mae import ViTMAEModel
# from tem_dataloader import MultimodalDatasetPerTrajectory
from torch.utils.data import DataLoader

from d3rlpy.algos import CQL
from d3rlpy.dataset import Episode, MDPDataset, Transition
wandb.login() 

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yadix. Use `wandb login --relogin` to force relogin


True

In [2]:
from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
import torch
# call CustomViT
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEConfig
from PIL import Image

# output_dir='/home/tyz/Desktop/11_777/camelmera/weights'
trained_model_name = 'multimodal'
output_dir='/home/ubuntu/weights/' + trained_model_name

# Initialize a new CustomViTMAE model
model_name = "facebook/vit-mae-base"

vit_config = ViTMAEConfig.from_pretrained(model_name)
vit_config.output_hidden_states=True
vit_model = CustomViT.from_pretrained(model_name,config=vit_config)


model_name = "facebook/vit-mae-base"

config = ViTMAEConfig.from_pretrained(model_name)
config.output_hidden_states=True

# load from pretrained model and replace the original encoder with custom encoder
custom_model = CustomViTMAE.from_pretrained("facebook/vit-mae-base",config=config)
custom_model.vit = vit_model

# Load the state_dict from the saved model
state_dict = torch.load(f"{output_dir}/pytorch_model.bin")

# Apply the state_dict to the custom_model
custom_model.load_state_dict(state_dict)

Some weights of the model checkpoint at facebook/vit-mae-base were not used when initializing CustomViT: ['decoder.decoder_layers.1.output.dense.bias', 'decoder.decoder_layers.4.attention.attention.key.bias', 'decoder.decoder_layers.3.layernorm_before.weight', 'decoder.decoder_layers.5.output.dense.weight', 'decoder.decoder_layers.6.attention.attention.query.weight', 'decoder.decoder_layers.4.attention.attention.query.bias', 'decoder.decoder_layers.1.attention.attention.key.bias', 'decoder.decoder_layers.7.layernorm_after.bias', 'decoder.decoder_layers.6.output.dense.weight', 'decoder.decoder_layers.1.attention.attention.query.weight', 'decoder.decoder_layers.0.layernorm_before.weight', 'decoder.decoder_layers.1.layernorm_after.bias', 'decoder.decoder_layers.4.intermediate.dense.bias', 'decoder.decoder_layers.1.attention.attention.value.weight', 'decoder.decoder_layers.4.layernorm_after.weight', 'decoder.decoder_layers.1.intermediate.dense.bias', 'decoder.decoder_layers.0.attention.out

<All keys matched successfully>

In [3]:
from tem_dataloader import MultimodalDatasetPerTrajectory
# environment_name = 'AmericanDinerExposure'
# environemnt_directory = f'/media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/{environment_name}/Data_easy'
# my_dataset = MultimodalDatasetPerTrajectory(environemnt_directory)
environment_name = 'AbandonedCableExposure'
environemnt_directory = f'/mnt/data/tartanairv2filtered/{environment_name}/Data_easy'
OBSERVATION_SIZE = 768
ACTION_SIZE = 7
BATCH_SIZE = 64

for folder in os.listdir(environemnt_directory):
    trajectory_folder_path = os.path.join(environemnt_directory, folder)
    if not os.path.isdir(trajectory_folder_path):
        continue
    my_dataset = MultimodalDatasetPerTrajectory(trajectory_folder_path)
    train_dataloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=False)
    break

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Processing folder: /mnt/data/tartanairv2filtered/AbandonedCableExposure/Data_easy/P001
Number of images: 1183
Number of depth: 1183
Number of lidar: 1183
Number of pose: 1183


In [4]:
import numpy as np

def reward_function(state_embedding, goal_embedding, threshold=0.05, goal_reward=100):
    distance = np.linalg.norm(state_embedding - goal_embedding)

    if distance <= threshold:
        # Give a large positive reward when the goal is reached
        reward = goal_reward
    else:
        # Give a negative reward proportional to the distance otherwise
        reward = -distance

    return reward

In [5]:
import functools
import numpy as np
import torch

# Initialize empty arrays for observations, actions, rewards, and terminals
all_observations = np.empty((0, OBSERVATION_SIZE))
all_actions = np.empty((0, ACTION_SIZE))
all_rewards = np.empty(0)
all_terminals = np.empty(0, dtype=bool)

for batch_idx, data in enumerate(train_dataloader):
    # get embedding
    custom_model.cuda()
    custom_model.eval()
    pixel_values = data["pixel_values"].cuda()
    pixel_values1 = data["pixel_values1"].cuda()
    pixel_values2 = data["pixel_values2"].cuda()
    outputs = custom_model(pixel_values,pixel_values1,pixel_values2,noise=None)
    embedding = outputs.hidden_states[:,0,:]
    observation = embedding.cpu().detach().numpy()
    # get action
    pose = data["pose_values"]
    action = torch.diff(pose,axis = 0).numpy()
    action = np.concatenate((action, np.zeros((1,7))), axis=0)
    # get reward
    goal = observation[-1]
    partial_function = functools.partial(reward_function, goal_embedding=goal)
    reward = np.apply_along_axis(partial_function, 1, observation)
    # get terminals
    terminals = [False]*data.shape[0]
    terminals[-1]=True
    terminals = np.array(terminals)

    # Concatenate observations, actions, rewards, and terminals
    all_observations = np.vstack((all_observations, observation))
    all_actions = np.vstack((all_actions, action))
    all_rewards = np.hstack((all_rewards, reward))
    all_terminals = np.hstack((all_terminals, terminals))

print("All observations shape:", all_observations.shape)
print("All actions shape:", all_actions.shape)
print("All rewards shape:", all_rewards.shape)
print("All terminals shape:", all_terminals.shape)

OutOfMemoryError: CUDA out of memory. Tried to allocate 152.00 MiB (GPU 0; 14.61 GiB total capacity; 10.91 GiB already allocated; 129.19 MiB free; 11.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
'''
Args:
        observations (numpy.ndarray): N-D array. If the
            observation is a vector, the shape should be
            `(N, dim_observation)`. If the observations is an image, the shape
            should be `(N, C, H, W)`.
        actions (numpy.ndarray): N-D array. If the actions-space is
            continuous, the shape should be `(N, dim_action)`. If the
            action-space is discrete, the shape should be `(N,)`.
        rewards (numpy.ndarray): array of scalar rewards. The reward function
            should be defined as :math:`r_t = r(s_t, a_t)`.
        terminals (numpy.ndarray): array of binary terminal flags.
        episode_terminals (numpy.ndarray): array of binary episode terminal
            flags. The given data will be splitted based on this flag.
            This is useful if you want to specify the non-environment
            terminations (e.g. timeout). If ``None``, the episode terminations
            match the environment terminations.
        discrete_action (bool): flag to use the given actions as discrete
            action-space actions. If ``None``, the action type is automatically
            determined.
    '''
cql_dataset = MDPDataset(observations=observation,\
                         actions=action,\
                         rewards=np.zeros(32),\
                         terminals=np.zeros(32),\
                         episode_terminals=np.array([False]*embedding.shape[0]))

In [ ]:
from d3rlpy.algos import CQL

# setup CQL algorithm
cql = CQL(use_gpu=False)

# split train and test episodes
# train_episodes, test_episodes = train_test_split(cql_dataset, test_size=0.25)

# start training
cql.fit(cql_dataset,
        eval_episodes=None,
        n_epochs=1,
        scorers=None)